In [30]:
from PIL import Image, ImageOps
import os
import shutil
from moviepy.editor import (
    ImageSequenceClip,
    concatenate_videoclips,
    AudioFileClip,
    vfx
)

## Upload the images and music and organize files

In [13]:
from google.colab import files
uploaded = files.upload()

In [14]:
import os
import shutil

# Create 'images' folder
os.makedirs("images", exist_ok=True)

# Move uploaded image files into 'images' folder
for filename in uploaded:
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        shutil.move(filename, os.path.join("images", filename))


In [15]:
music_file = None
for filename in uploaded:
    if filename.lower().endswith('.mp3'):
        music_file = filename
        break

## Script

In [31]:
def resize_and_pad(img, target_size):
    """Resize and pad image to target size while maintaining aspect ratio."""
    img = ImageOps.contain(img, target_size)
    background = Image.new("RGB", target_size, (0, 0, 0))
    offset = ((target_size[0] - img.size[0]) // 2, (target_size[1] - img.size[1]) // 2)
    background.paste(img, offset)
    return background

def create_zoom_frames(image_path, output_folder, zoom_start, zoom_end, steps=150, target_size=(1920, 1080)):
    os.makedirs(output_folder, exist_ok=True)
    base_img = Image.open(image_path)
    base_img = resize_and_pad(base_img, target_size)
    w, h = base_img.size

    for i in range(steps):
        zoom = zoom_start + (zoom_end - zoom_start) * (i / (steps - 1))

        # Subpixel cropping box
        crop_w = w / zoom
        crop_h = h / zoom
        left = (w - crop_w) / 2
        top = (h - crop_h) / 2
        right = left + crop_w
        bottom = top + crop_h

        # Crop and resize
        cropped = base_img.crop((left, top, right, bottom)).resize((w, h), Image.LANCZOS)
        cropped.save(os.path.join(output_folder, f"frame_{i:03d}.png"))


def generate_zoom_clip(image_path, mode, duration_seconds, fps, temp_folder, target_size):
    steps = duration_seconds * fps
    zoom_start, zoom_end = (1.0, 1.1) if mode == "in" else (1.1, 1.0)

    create_zoom_frames(image_path, temp_folder, zoom_start, zoom_end, steps, target_size)

    frame_files = sorted([
        os.path.join(temp_folder, f)
        for f in os.listdir(temp_folder)
        if f.endswith(".png")
    ])
    clip = ImageSequenceClip(frame_files, fps=fps).set_duration(duration_seconds)
    clip = clip.fx(vfx.fadein, 1).fx(vfx.fadeout, 1)
    return clip  # Don't delete the folder yet


def create_combined_zoom_video_from_folder(
    image_folder,
    output_video="zoom_sequence.mp4",
    music_path=None,
    duration_seconds=5,
    fps=30,
    platform="youtube"
):
    # Set target size based on platform
    if platform == "youtube":
        target_size = (1920, 1080)
    elif platform == "tiktok":
        target_size = (1080, 1920)
    else:
        raise ValueError("Platform must be 'youtube' or 'tiktok'")

    image_paths = sorted([
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ])

    if not image_paths:
        raise ValueError("No images found in the folder.")

    clips = []
    for idx, image_path in enumerate(image_paths):
        mode = "in" if idx % 2 == 0 else "out"
        print(f"Processing image {idx+1}/{len(image_paths)} with zoom {mode}")
        clip = generate_zoom_clip(
            image_path=image_path,
            mode=mode,
            duration_seconds=duration_seconds,
            fps=fps,
            temp_folder=f"temp_frames_{idx}",
            target_size=target_size
        )
        clips.append(clip)

    final_clip = concatenate_videoclips(clips, method="compose")

    if music_path and os.path.exists(music_path):
        audio = AudioFileClip(music_path)
        video_duration = final_clip.duration

        if audio.duration < video_duration:
            loops = int(video_duration // audio.duration) + 1
            audio = concatenate_videoclips([audio] * loops).subclip(0, video_duration)
        else:
            audio = audio.subclip(0, video_duration)

        audio = audio.audio_fadeout(2)
        final_clip = final_clip.set_audio(audio)

    final_clip.write_videofile(
    output_video,
    codec="libx264",
    audio=bool(music_path),
    audio_codec="aac"
)


## Run the script

In [32]:
# run in colab
create_combined_zoom_video_from_folder(
    image_folder="images",
    output_video="zoom_sequence.mp4",
    music_path="background_music.mp3",    # Set to None if no music
    duration_seconds=5,
    fps=30,
    platform="youtube"        # or "tiktok"
)

Processing image 1/4 with zoom in
Processing image 2/4 with zoom out
Processing image 3/4 with zoom in
Processing image 4/4 with zoom out
Moviepy - Building video zoom_sequence.mp4.
MoviePy - Writing audio in zoom_sequenceTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video zoom_sequence.mp4



Moviepy - Done !
Moviepy - video ready zoom_sequence.mp4


## Download

In [33]:
from google.colab import files
files.download("zoom_sequence.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === USAGE ===
create_combined_zoom_video_from_folder(
    image_folder="images",                 # Folder with your images
    output_video="zoom_sequence.mp4",      # Output video file
    music_path="background.mp3",           # Optional music file; None if no music
    duration_seconds=5,                    # Duration per image
    fps=30,                                # Frames per second
    platform="youtube"                     # 'youtube' (1920x1080) or 'tiktok' (1080x1920)
)

## Clean up

In [11]:
def cleanup_temp_folders(prefix="temp_frames_"):
    import shutil
    import os
    for folder in os.listdir():
        if folder.startswith(prefix) and os.path.isdir(folder):
            print(f"Deleting folder: {folder}")
            shutil.rmtree(folder)

    final_clip.write_videofile(output_video, codec="libx264", audio=bool(music_path))

    # 🧹 Cleanup temp frame folders after rendering
    cleanup_temp_folders()
